In [147]:
%run Vector.ipynb

In [32]:
np.set_printoptions(threshold=10) #np.inf

In [149]:
with open('uva_json_file', 'r') as f:
    olddata = json.load(f)

## Input 

Dictionary:

    Language (int):
        0 = Nederlands       
        1 = Engels
              
    Type (int):  
        0 = Onbekend      
        1 = Wie        
        2 = Wat        
        3 = Waar        
        4 = Waarom        
        5 = Wanneer        
        6 = Hoe        
        7 = Welke        
        
    Level (str):        
        Studie        
        Faculteit        
        UvA        
        None        
        
    Keywords (set):        
        {<keyword_1>, ..., <keyword_n>}        
        
    Source (str):
        <Oorspronkelijke zin>

We assume that for this specific problem (only searching the tree we have established) we actually only need the keywords provided from the input query. The reason for this is that the keywords of the articles we have are a mix of both Dutch and English and will therefore automatically return the right URL. The type is irrelevant for this problem as we only have access to the keywords of the articles because we have not scraped the entire texts. Therefore, we are unable to parse what kind of question a person is asking, this will be handled in our other solution of using Watson to find the right text. 
The source is also not relevant as we have the keywords necessary.

In [151]:
query = {'Language':0, 
         'Type':0, 
         'Level':None, 
         'Keywords':set(['laptop', 'eisen', 'kunstmatige', 'intelligentie']), 
         'Source':'Wat zijn de laptop eisen voor kunstmatige intelligentie?'}

In [141]:
def leveltype(level, data):
    for faculty, studies in data.items():
        if faculty == level:
            return 'fac'
        else:
            for study, az_links in studies.items():
                if study == level:
                    return 'stud'
    return None

In [142]:
def lvlkey(link, olddata):
    for faculty, studies in olddata.items():
        for study, az_links in studies.items():
            for article, keywords in az_links.items():
                if link == article:
                    return study, keywords
    return None

In [143]:
def analyse_query(query, data):
    keywords = list(query.get('Keywords'))
    level = query.get('Level')
    lvltype = leveltype(level, data)
    queryvec = vector(keywords, generalvec)
    return queryvec, level, lvltype, keywords

In [144]:
def search_tree(data, queryvec, level, leveltype):
    topdict = {}
    value = []
    threshold = 0        
    if leveltype == 'fac':
        for faculty, studies in data.items():
            if faculty == level:
                for study, az_links in studies.items():
                    for article, articlevec in az_links.items():
                        cosinesim = 1 - cs.distance.cosine(articlevec, queryvec)
                        if cosinesim > threshold:
                            topdict.update({article: cosinesim})
    elif leveltype == 'stud':
        for faculty, studies in data.items():
            for study, az_links in studies.items():
                if study == level:
                    for article, articlevec in az_links.items():
                        cosinesim = 1 - cs.distance.cosine(articlevec, queryvec)
                        if cosinesim > threshold:
                            topdict.update({article: cosinesim})
    else:
        for faculty, studies in data.items():
                for study, az_links in studies.items():
                    for article, articlevec in az_links.items():
                        cosinesim = 1 - cs.distance.cosine(articlevec, queryvec)
                        if cosinesim > threshold:
                            topdict.update({article: cosinesim})
    toplinks = sorted(topdict, key=topdict.get, reverse=True)
    return toplinks, topdict

In [145]:
def topresults(olddata, toplinks, topdict, n):
    topndicts = []
    for toplink in toplinks[:n]:
        level, keywords = lvlkey(toplink, olddata)
        for link, score in topdict.items():
            if toplink == link:
                topndicts.append({'URL':link, 'Score':score, 'Level':level, 'Keywords':keywords})
    return topndicts

In [153]:
def main(query, data, olddata, n):
    queryvec, level, lvltype, keywords = analyse_query(query, data)
    toplinks, topdict = search_tree(data, queryvec, level, lvltype)
    topndicts = topresults(olddata, toplinks, topdict, n)
    return topndicts

print(main(query, data, olddata, 5))

[{'URL': 'http://www.student.uva.nl/sgpl/content/az/vak-en-tentamenaanmelding-sgp/ingangseisen/ingangseisen-kopie.html', 'Score': 0.25, 'Level': 'Sociale geografie en Planologie (bachelor)', 'Keywords': ['maatschappij- en gedragswetenschappen,studieprogramma,vakaanmelding,sociale geografie en planologie (bachelor),eisen,voorwaarden,in- en uitschrijven,instroom']}, {'URL': 'http://www.student.uva.nl/inc/content/az/laptop-minimumeisen/laptop-minimumeisen.html', 'Score': 0.25, 'Level': 'Informatica (bachelor)', 'Keywords': ['minimumeisen,studenten,ict en faciliteiten,natuurwetenschappen, wiskunde en informatica,computereisen,laptop,informatica']}, {'URL': 'http://www.student.uva.nl/sgpl/content/az/ingangseisen/ingangseisen.html', 'Score': 0.23570226039551578, 'Level': 'Sociale geografie en Planologie (bachelor)', 'Keywords': ['maatschappij- en gedragswetenschappen,studieprogramma,inschrijving en toelating,vakaanmelding,sociale geografie en planologie (bachelor),eisen,voorwaarden,in- en ui